# 모든 데이터에서 위도경도 구하기

- 모든 데이터의 시군구, 번지의 결측치 크롤링
- 시군구, 번지 기준 위도와 경도 구하기
- 위도, 경도 기준으로 주변 50m, 100m, 500m, 1000m 기준 역과 버스정거정의 개수 구하기

- 좌표Y : 위도
- 좌표X : 경도

In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf", # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

from geopy.geocoders import Nominatim
import googlemaps

from haversine import haversine

In [2]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '/root/data/train.csv'
test_path  = '/root/data/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

In [48]:
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


In [3]:
dt_copy = dt.copy()

## 위도경도를 찾아야 하는 주소들만 추출하여 작업

In [4]:
import requests
a = dt_copy[dt_copy['좌표X'].isna()]
a['new_address'] = a['시군구'] + " " + a['도로명']
a['old_address'] = a['시군구'] + ' ' + a['번지']

In [5]:
need_tofind_newaddr = a[a['좌표X'].isna()]['new_address'].value_counts().index #위도 경도가 필요한 주소들
need_tofind_oldaddr = a[a['좌표X'].isna()]['old_address'].value_counts().index

In [9]:
need_tofind_newaddr

Index(['서울특별시 송파구 신천동 올림픽로 435', '서울특별시 강북구 미아동 솔샘로 174',
       '서울특별시 성북구 돈암동 성북로4길 52', '서울특별시 강동구 둔촌동 양재대로 1340',
       '서울특별시 관악구 봉천동 성현로 80', '서울특별시 송파구 가락동 송파대로 345',
       '서울특별시 송파구 잠실동 올림픽로 135', '서울특별시 송파구 방이동 양재대로 1218',
       '서울특별시 강동구 암사동 상암로 11', '서울특별시 금천구 시흥동 금하로 816',
       ...
       '서울특별시 구로구 항동 항동로 60', '서울특별시 금천구 시흥동 시흥대로 224',
       '서울특별시 동대문구 이문동 이문로35아길 16', '서울특별시 동대문구 전농동 전농로37길 35',
       '서울특별시 동대문구 휘경동 망우로12길 25', '서울특별시 동작구 상도동 국사봉6길 58',
       '서울특별시 서대문구 남가좌동 거북골로14길 32', '서울특별시 서초구 방배동 방배로32길 112',
       '서울특별시 송파구 송파동 오금로32길 35', '서울특별시 용산구 한강로2가  '],
      dtype='object', length=8500)

In [10]:
lat_new = []
lon_new = []
is_find_new = []

In [13]:
lat_old = []
lon_old = []
is_find_old = []

In [11]:
# 신주소로 검색
tmp = 0
url = "https://dapi.kakao.com/v2/local/search/address.json" #요청할 url 주소
headers = {"Authorization": 'KakaoAK 7e13c0b7854960718bdc7832993d1a28'} #REST API 키(유효한 키)

for new_addr in need_tofind_newaddr:

    # if a.loc[idx, 'cant_find_xy'] == 1:
    #     continue

    
    query = {'query': new_addr } #입력할 주소
    result = requests.get(url,
                        headers=headers,
                        data=query).json() #카카오 API 요청
    try:
        lat_new.append(result['documents'][0]['y']) # 위도
        lon_new.append(result['documents'][0]['x']) # 경도
        is_find_new.append(None)
        print(new_addr + ' 성공'+str(tmp+1))
    except:
        lat_new.append(None)
        lon_new.append(None)
        is_find_new.append(1)
        print(new_addr + ' 실패'+str(tmp+1))

    tmp += 1

서울특별시 송파구 신천동 올림픽로 435 성공
서울특별시 강북구 미아동 솔샘로 174 성공
서울특별시 성북구 돈암동 성북로4길 52 성공
서울특별시 강동구 둔촌동 양재대로 1340 실패
서울특별시 관악구 봉천동 성현로 80 성공
서울특별시 송파구 가락동 송파대로 345 성공
서울특별시 송파구 잠실동 올림픽로 135 성공
서울특별시 송파구 방이동 양재대로 1218 성공
서울특별시 강동구 암사동 상암로 11 성공
서울특별시 금천구 시흥동 금하로 816 성공
서울특별시 중구 신당동 다산로 32 성공
서울특별시 강동구 암사동 고덕로 131 성공
서울특별시 도봉구 창동 덕릉로66길 17 성공
서울특별시 노원구 월계동 마들로 31 성공
서울특별시 노원구 상계동 노원로 532 성공
서울특별시 노원구 중계동 중계로 230 성공
서울특별시 강동구 명일동 고덕로 210 성공
서울특별시 송파구 잠실동 잠실로 62 성공
서울특별시 송파구 잠실동 송파대로 567 성공
서울특별시 도봉구 도봉동 마들로 859-19 성공
서울특별시 노원구 공릉동 공릉로34길 62 성공
서울특별시 송파구 가락동 동남로 193 성공
서울특별시 노원구 상계동 동일로227길 86 성공
서울특별시 성동구 행당동 행당로 82 성공
서울특별시 관악구 봉천동 은천로 93 성공
서울특별시 성동구 행당동 행당로 79 성공
서울특별시 노원구 중계동 한글비석로14길 8 성공
서울특별시 양천구 신정동 목동동로 130 성공
서울특별시 노원구 월계동 초안산로1길 18 성공
서울특별시 노원구 상계동 노해로 508 성공
서울특별시 구로구 신도림동 신도림로 16 성공
서울특별시 노원구 상계동 노원로 564 성공
서울특별시 중랑구 신내동 신내로19길 42 성공
서울특별시 동대문구 휘경동 한천로 248 성공
서울특별시 노원구 하계동 섬밭로 229 성공
서울특별시 구로구 개봉동 개봉로20길 6 성공
서울특별시 노원구 하계동 섬밭로 196 성공
서울특별시 관악구 신림동 난곡로 55 성공
서울특별시 강서구 화곡동 강서로 266 성공
서울특별시 

In [14]:
# 구주소로 검색
tmp = 0
url = "https://dapi.kakao.com/v2/local/search/address.json" #요청할 url 주소
headers = {"Authorization": 'KakaoAK 7e13c0b7854960718bdc7832993d1a28'} #REST API 키(유효한 키)

for old_addr in need_tofind_oldaddr:

    # if a.loc[idx, 'cant_find_xy'] == 1:
    #     continue

    
    query = {'query': old_addr } #입력할 주소
    result = requests.get(url,
                        headers=headers,
                        data=query).json() #카카오 API 요청
    try:
        lat_old.append(result['documents'][0]['y']) # 위도
        lon_old.append(result['documents'][0]['x']) # 경도
        is_find_old.append(None)
        print(old_addr + ' 성공'+str(tmp+1))
    except:
        lat_old.append(None)
        lon_old.append(None)
        is_find_old.append(1)
        print(old_addr + ' 실패'+str(tmp+1))

    tmp += 1

서울특별시 송파구 신천동 17 성공1
서울특별시 강북구 미아동 1353 성공2
서울특별시 동작구 사당동 105 성공3
서울특별시 성북구 돈암동 609-1 성공4
서울특별시 노원구 중계동 502-1 성공5
서울특별시 관악구 봉천동 1712 성공6
서울특별시 송파구 가락동 479 실패7
서울특별시 송파구 잠실동 22 성공8
서울특별시 도봉구 방학동 271-1 성공9
서울특별시 송파구 방이동 89 성공10
서울특별시 강동구 암사동 509 성공11
서울특별시 금천구 시흥동 1013 성공12
서울특별시 노원구 월계동 13 성공13
서울특별시 중구 신당동 844 성공14
서울특별시 노원구 상계동 639 성공15
서울특별시 강동구 암사동 414-2 성공16
서울특별시 도봉구 창동 38 성공17
서울특별시 노원구 월계동 18 성공18
서울특별시 노원구 상계동 670 성공19
서울특별시 강남구 개포동 12 성공20
서울특별시 노원구 중계동 359-1 성공21
서울특별시 강동구 명일동 15 성공22
서울특별시 송파구 잠실동 35 성공23
서울특별시 송파구 잠실동 27 성공24
서울특별시 도봉구 도봉동 30-1 성공25
서울특별시 노원구 공릉동 81 성공26
서울특별시 송파구 가락동 140 성공27
서울특별시 강동구 둔촌동 170-1 성공28
서울특별시 노원구 상계동 624 성공29
서울특별시 성동구 행당동 346 성공30
서울특별시 관악구 봉천동 1718 성공31
서울특별시 노원구 상계동 626 성공32
서울특별시 노원구 상계동 691 성공33
서울특별시 성동구 행당동 347 성공34
서울특별시 노원구 상계동 720 성공35
서울특별시 노원구 중계동 369 성공36
서울특별시 양천구 신정동 329 성공37
서울특별시 노원구 월계동 556 성공38
서울특별시 구로구 신도림동 642 성공39
서울특별시 노원구 상계동 666 성공40
서울특별시 중랑구 신내동 650 성공41
서울특별시 동대문구 휘경동 57 성공42
서울특별시 노원구 하계동 271-3 성공43
서울특별시 구로구 개봉동

In [15]:
addr_list_new = {
    'new_addr' : need_tofind_newaddr,
    'lat' : lat_new,
    'lon' : lon_new,
    'is_find' : is_find_new
    }

In [16]:
addr_list_old = {
    'old_addr' : need_tofind_oldaddr,
    'lat' : lat_old,
    'lon' : lon_old,
    'is_find' : is_find_old
    }

In [17]:
found_newaddr = pd.DataFrame(addr_list_new)
found_oldaddr = pd.DataFrame(addr_list_old)

In [21]:
found_newaddr

,new_addr,lat,lon,is_find
0,서울특별시 송파구 신천동 올림픽로 435,37.5213592954462,127.104971745978,NaN
1,서울특별시 강북구 미아동 솔샘로 174,37.6176383628418,127.010599691076,NaN
2,서울특별시 성북구 돈암동 성북로4길 52,37.5944725333435,127.012706037821,NaN
3,서울특별시 강동구 둔촌동 양재대로 1340,None,None,1.0
4,서울특별시 관악구 봉천동 성현로 80,37.4898474387668,126.952985510916,NaN
...,...,...,...,...
8495,서울특별시 동작구 상도동 국사봉6길 58,37.4947953102949,126.929732008605,NaN
8496,서울특별시 서대문구 남가좌동 거북골로14길 32,None,None,1.0
8497,서울특별시 서초구 방배동 방배로32길 112,37.4926079942098,126.997550607148,NaN
8498,서울특별시 송파구 송파동 오금로32길 35,37.5032486675314,127.119035919533,NaN


In [22]:
found_newaddr[found_newaddr['is_find']==1]

,new_addr,lat,lon,is_find
3,서울특별시 강동구 둔촌동 양재대로 1340,None,None,1.0
108,서울특별시 강동구 상일동 고덕로80길 13,None,None,1.0
121,서울특별시 강동구 명일동 양재대로 1656,None,None,1.0
146,서울특별시 강남구 개포동 언주로 21,None,None,1.0
174,서울특별시 강동구 고덕동 고덕로 313,None,None,1.0
...,...,...,...,...
8480,서울특별시 마포구 공덕동 18,None,None,1.0
8481,서울특별시 서대문구 냉천동 천중로56길 41,None,None,1.0
8487,서울특별시 양천구 목동 남부순환로83길 47,None,None,1.0
8492,서울특별시 동대문구 이문동 이문로35아길 16,None,None,1.0


In [23]:
found_oldaddr

,old_addr,lat,lon,is_find
0,서울특별시 송파구 신천동 17,37.5205366173754,127.10615089337,NaN
1,서울특별시 강북구 미아동 1353,37.6179760334621,127.01263074872,NaN
2,서울특별시 동작구 사당동 105,37.4896946254326,126.975650061703,NaN
3,서울특별시 성북구 돈암동 609-1,37.5941783218836,127.010321322145,NaN
4,서울특별시 노원구 중계동 502-1,37.6443713328952,127.063281683752,NaN
...,...,...,...,...
8187,서울특별시 강남구 역삼동 682-1,37.5093407737874,127.042807378726,NaN
8188,서울특별시 중랑구 신내동 1,None,None,1.0
8189,서울특별시 강남구 역삼동 728-6,37.4976861985099,127.039648336329,NaN
8190,서울특별시 강남구 청담동 90-13,37.5251962776812,127.042541084136,NaN


In [24]:
found_oldaddr[found_oldaddr['is_find']==1]

,old_addr,lat,lon,is_find
6,서울특별시 송파구 가락동 479,None,None,1.0
95,서울특별시 강동구 상일동 121,None,None,1.0
121,서울특별시 강동구 고덕동 212,None,None,1.0
127,서울특별시 강동구 명일동 294,None,None,1.0
219,서울특별시 강동구 고덕동 492,None,None,1.0
...,...,...,...,...
8152,서울특별시 성동구 하왕십리동 811,None,None,1.0
8153,서울특별시 동작구 신대방동 729-24,None,None,1.0
8172,서울특별시 용산구 문배동 133-3,None,None,1.0
8188,서울특별시 중랑구 신내동 1,None,None,1.0


## 주소별 위도 경도 데이터를 실제 데이터에 대입

In [39]:
dt_copy = dt.copy()

In [40]:
dt_copy['new_address'] = dt_copy['시군구'] + " " + dt_copy['도로명']
dt_copy['old_address'] = dt_copy['시군구'] + " " + dt_copy['번지']
dt_copy['cant_find_xy'] = np.nan
dt_copy.loc[3060,]

시군구                               서울특별시 강남구 삼성동
번지                                         19-4
본번                                         19.0
부번                                          4.0
아파트명                                         상아
전용면적(㎡)                                   91.54
계약년월                                     201801
계약일                                          26
층                                             1
건축년도                                       1981
도로명                                     삼성로 649
해제사유발생일                                     NaN
등기신청일자                                         
거래유형                                          -
중개사소재지                                        -
k-단지분류(아파트,주상복합등등)                          NaN
k-전화번호                                      NaN
k-팩스번호                                      NaN
단지소개기존clob                                  NaN
k-세대타입(분양형태)                                NaN
k-관리방식                                  

In [41]:
# 신주소기준 위도 경도 갱신
tmp = 0
for adr, lat, lon, is_find in zip(found_newaddr['new_addr'],found_newaddr['lat'], found_newaddr['lon'], found_newaddr['is_find']):
    # 만약 해당 주소의 위도 경도를 찾았다면 
    
    if is_find == 1:
        dt_copy.loc[(dt_copy['new_address'] == adr) & (dt_copy['좌표X'].isna()), ['cant_find_xy'] ] = is_find
    else:
        dt_copy.loc[(dt_copy['new_address'] == adr) & (dt_copy['좌표X'].isna()), ['좌표X'] ] = lon
        dt_copy.loc[(dt_copy['new_address'] == adr) & (dt_copy['좌표Y'].isna()), ['좌표Y'] ] = lat
    print(tmp+1)
    tmp+=1
    # print(adr, lat, lon, is_find)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [42]:
# 구주소기준 위도 경도 갱신
tmp =0
for adr, lat, lon, is_find in zip(found_oldaddr['old_addr'],found_oldaddr['lat'], found_oldaddr['lon'], found_oldaddr['is_find']):
    # 만약 해당 주소의 위도 경도를 찾았다면 
    if is_find == 1:
        dt_copy.loc[(dt_copy['old_address'] == adr) & (dt_copy['좌표X'].isna()), ['cant_find_xy'] ] = is_find
    else:
        dt_copy.loc[(dt_copy['old_address'] == adr) & (dt_copy['좌표X'].isna()), ['좌표X'] ] = lon
        dt_copy.loc[(dt_copy['old_address'] == adr) & (dt_copy['좌표Y'].isna()), ['좌표Y'] ] = lat
    print(tmp+1)
    tmp+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [43]:
dt.loc[3060,]

시군구                       서울특별시 강남구 삼성동
번지                                 19-4
본번                                 19.0
부번                                  4.0
아파트명                                 상아
전용면적(㎡)                           91.54
계약년월                             201801
계약일                                  26
층                                     1
건축년도                               1981
도로명                             삼성로 649
해제사유발생일                             NaN
등기신청일자                                 
거래유형                                  -
중개사소재지                                -
k-단지분류(아파트,주상복합등등)                  NaN
k-전화번호                              NaN
k-팩스번호                              NaN
단지소개기존clob                          NaN
k-세대타입(분양형태)                        NaN
k-관리방식                              NaN
k-복도유형                              NaN
k-난방방식                              NaN
k-전체동수                              NaN
k-전체세대수                             NaN


## 최종본 저장

In [57]:
dt_copy.to_csv('/root/data/train_make_lat_lon.csv', index=False)